## BERTopic

### Bibliotecas

In [1]:
#Bibliotecas

import pandas as pd
import re
from bertopic import BERTopic
import warnings
import nltk
from nltk.corpus import stopwords
import itertools
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import matplotlib
import matplotlib.pyplot as plt
from umap import UMAP

#Estilizar conteúdo
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

def estilo_tabelas(df, max_altura='300px', casas_decimais=2):
    return (
        df.style.set_table_styles(
            [
                {'selector': 'thead th', 'props': [('font-size', '12px'), ('text-align', 'center'), ('border-bottom', '2px solid #007BFF')]},
                {'selector': 'td', 'props': [('font-size', '10px'), ('text-align', 'center'), ('max-height', '40px'), ('white-space', 'nowrap'), ('text-overflow', 'ellipsis'), ('overflow', 'hidden'), ('max-width', '100px')]},
                {'selector': 'tr:nth-child(odd)', 'props': [('background-color', '#ffffff')]},
                {'selector': 'table', 'props': [('width', '90%'), ('margin-left', 'auto'), ('margin-right', 'auto'), ('border-collapse', 'collapse')]},
                {'selector': 'td, th', 'props': [('border', '1px solid #666')]},
            ]
        ).set_properties(
            **{'border-color': 'darkgray', 'border-style': 'solid', 'border-width': '1px'}
        ).set_table_attributes(
            f'style="height:auto; overflow:auto; max-height:{max_altura}; display:block;"'  
        ).format(
            precision=casas_decimais  
        )
    )

### Base de dados

In [2]:
object_columns = ['Protocolo_S2iD', 'Nome_Municipio', 'Sigla_UF', 'regiao',
                  'Setores Censitários', 'Status', 'DH_Descricao', 'DM_Descricao',
                  'DA_Descricao', 'DA_Polui/cont da água', 'DA_Polui/cont do ar',
                  'DA_Polui/cont do solo', 'DA_Dimi/exauri hídrico',
                  "DA_Incêndi parques/APA's/APP's", 'PEPL_Descricao', 'PEPR_Descricao',
                  'Categoria', 'Grupo', 'Subgrupo', 'Tipo', 'Subtipo']

dtype = {col: 'object' for col in object_columns}

df_eventos = pd.read_csv(
    "https://raw.githubusercontent.com/brunagmoura/PrevisorReconhecimento/refs/heads/main/df_eventos_desastres_rec_nrec.csv",
    sep=';',
    decimal=',',
    dtype=dtype)

colunas_descr = ['Status', 'DM_Descricao', 'DA_Descricao', 'PEPL_Descricao', 'PEPR_Descricao', 'DH_Descricao']

df_eventos_descr = df_eventos[colunas_descr].fillna(' ').copy()

### Pré-processamento dos dados

In [3]:
# Baixar stopwords em português
nltk.download('stopwords')
stopwords_pt = stopwords.words('portuguese')

# Função para pré-processamento de texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remover números
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remover caracteres especiais
    text = ' '.join([word for word in text.split() if word not in stopwords_pt])  # Remover stopwords
    return text

# Transformando descrições em listas e removendo strings vazias
lista_dm = [desc for desc in df_eventos_descr['DM_Descricao'].values.tolist() if desc]
lista_da = [desc for desc in df_eventos_descr['DA_Descricao'].values.tolist() if desc]
lista_pepl = [desc for desc in df_eventos_descr['PEPL_Descricao'].values.tolist() if desc]
lista_pepr = [desc for desc in df_eventos_descr['PEPR_Descricao'].values.tolist() if desc]
lista_dh = [desc for desc in df_eventos_descr['DH_Descricao'].values.tolist() if desc]

# Filtrando eventos "Reconhecidos" e "Não reconhecidos"
df_reconhecido = df_eventos_descr[df_eventos_descr['Status'] == 'Reconhecido']
df_nao_reconhecido = df_eventos_descr[df_eventos_descr['Status'] == 'Não reconhecido']

# Criar listas para reconhecidos e não reconhecidos, removendo entradas vazias
lista_dm_r = [desc for desc in df_reconhecido['DM_Descricao'].values.tolist() if desc]
lista_da_r = [desc for desc in df_reconhecido['DA_Descricao'].values.tolist() if desc]
lista_pepl_r = [desc for desc in df_reconhecido['PEPL_Descricao'].values.tolist() if desc]
lista_pepr_r = [desc for desc in df_reconhecido['PEPR_Descricao'].values.tolist() if desc]
lista_dh_r = [desc for desc in df_reconhecido['DH_Descricao'].values.tolist() if desc]

lista_dm_n = [desc for desc in df_nao_reconhecido['DM_Descricao'].values.tolist() if desc]
lista_da_n = [desc for desc in df_nao_reconhecido['DA_Descricao'].values.tolist() if desc]
lista_pepl_n = [desc for desc in df_nao_reconhecido['PEPL_Descricao'].values.tolist() if desc]
lista_pepr_n = [desc for desc in df_nao_reconhecido['PEPR_Descricao'].values.tolist() if desc]
lista_dh_n = [desc for desc in df_nao_reconhecido['DH_Descricao'].values.tolist() if desc]

# Concatenando listas usando itertools.chain
lista_tudo = list(itertools.chain(lista_dm, lista_da, lista_pepl, lista_pepr, lista_dh))
lista_rec_ = list(itertools.chain(lista_dm_r, lista_da_r, lista_pepl_r, lista_pepr_r, lista_dh_r))
lista_nao_ = list(itertools.chain(lista_dm_n, lista_da_n, lista_pepl_n, lista_pepr_n, lista_dh_n))

# Aplicar o pré-processamento
lista_rec = [preprocess_text(text) for text in lista_rec_ if text]
lista_nao = [preprocess_text(text) for text in lista_nao_ if text]


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bruna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Modelo BERT

#### Tópicos dos eventos reconhecidos

In [ ]:
# Eventos reconhecidos
topic_model_rec = BERTopic()
topics_rec, probabilities_rec = topic_model_rec.fit_transform(lista_rec)

# Exibir tópicos identificados para eventos reconhecidos
topic_info_rec = topic_model_rec.get_topic_info()
print("\nTópicos identificados (Reconhecidos):")
print(topic_info_rec)

In [ ]:
# Exibir as palavras-chave de cada tópico reconhecido
for topic_num in topic_info_rec['Topic'].unique():
    if topic_num != -1:  # Ignorar outliers
        print(f"\nTópico {topic_num} (Reconhecidos): {topic_model_rec.get_topic(topic_num)}")

#### Tópicos dos eventos não reconhecidos

In [ ]:
topic_model_nao = BERTopic()
topics_nao, probabilities_nao = topic_model_nao.fit_transform(lista_nao)

# Eventos não reconhecidos
topic_info_nao = topic_model_nao.get_topic_info()
print("\nTópicos identificados (Não reconhecidos):")
print(topic_info_nao)

In [ ]:
# Exibir as palavras-chave de cada tópico não reconhecido
for topic_num in topic_info_nao['Topic'].unique():
    if topic_num != -1:  # Ignorar outliers
        print(f"\nTópico {topic_num} (Não reconhecidos): {topic_model_nao.get_topic(topic_num)}")

### Resultados

#### Eventos reconhecidos

In [ ]:
# Visualização dos tópicos
print("\nVisualizando tópicos dos eventos reconhecidos:")
topic_model_rec.visualize_topics().show()

#### Eventos não reconhecidos

In [ ]:
print("\nVisualizando tópicos dos eventos não reconhecidos:")
topic_model_nao.visualize_topics().show()

Comparar os que erraram e os que acertaram